In [136]:
import os, sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
student_life_path = module_path + "/student_life"
physnet_path = module_path + "/Physionet_rev"
print(student_life_path)
print(physnet_path)

sys.path.append(physnet_path)
sys.path.append(student_life_path)

import pandas as pd
import numpy as np
import pickle
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
print(torch.cuda.is_available())
print(torch.version.cuda)

import tbm as tbm
from net_dbm import RNN_osaka


import evaluate_plot as eval_plot
import batchify as batchify
from sklearn.metrics import precision_recall_fscore_support
import importlib
import src.utils.student_utils as student_utils
import src.definitions as definitions
from IPython.display import display
importlib.reload(student_utils)

# 1 starts the process on GPU-0
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.__version__)
%matplotlib inline
print(eval_plot)
print(sys.version_info)

/Users/nsimsiri/Documents/code/ml/MultiRes/student_life
/Users/nsimsiri/Documents/code/ml/MultiRes/Physionet_rev
False
None
1.0.1.post2
<module 'evaluate_plot' from '/Users/nsimsiri/Documents/code/ml/MultiRes/Physionet_rev/evaluate_plot.py'>
sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)


In [ ]:
import subprocess

p = subprocess.Popen(["nvcc", "--version"], stdout=subprocess.PIPE)

print(p.communicate())


In [35]:
w = torch.Tensor([0, 4, 8, 2]) # Create a tensor of weights
r = torch.multinomial(w, 4, replacement=True) # Size 4: 3, 2, 1, 2
print(r)

tensor([2, 2, 3, 2])


In [156]:
x = [1,2,3]
for i in range(10):
    k = np.random.choice(x, size=3, replace=False).tolist()
    print(k.pop(),)
    print(k.pop(),)
    print(k.pop())


3
2
1
1
2
3
2
3
1
2
3
1
2
3
1
3
2
1
1
3
2
3
2
1
3
1
2
1
3
2


In [135]:
import scipy
import numpy as np
import scipy.stats as stats
N = 5
T = 1000
a = [0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0]
b = [1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,0,1,0,0,1,1,1,1,0,1,1,1,1,0]
def bootstrap(A, B):
    grams = []
    for t in range(T):
        sample = np.random.choice(A, N)
        sample2 = np.random.choice(B, N)
        gram = np.mean(sample)
        gram2 = np.mean(sample2)
        grams.append(np.abs(gram-gram2))
    grams.sort()
    std = stats.sem(grams)
    mu = np.mean(grams)
    print(np.mean(A), mu, std)
    conf = stats.t.interval(0.95, len(grams)-1, loc=mu, scale=std)
    print(conf)
bootstrap(a, b)

0.3333333333333333 0.4505999999999999 0.008071231924854148
(0.4347614869514347, 0.4664385130485651)


In [94]:
a1 = mean_confidence_interval(a)
print(a1)


(0.3333333333333333, 0.1542987942083919, 0.5123678724582748)


In [84]:
a2 = bootstrap(a)
# print(a2)

0.3333333333333333 0.33205714285714283


In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h
